In [257]:
import pandas as pd 
import matplotlib as mpl 
import matplotlib.pyplot as plt 
import seaborn as sns 
import scipy.stats as stats
import plotly.express as px
from datetime import datetime

mpl.rc('font',family='Malgun Gothic')

In [259]:
df_sales = pd.read_csv('Sales_Data05.csv')
df_member = pd.read_csv('Member_Data.csv',encoding='cp949')
df_product = pd.read_csv('Product_Data.csv')

In [260]:
df_sales.head()

,회원번호,회원상태,구매수량,구매금액,주문일시,배송시작일,배송완료일,사용 적립금,사용 포인트 네이버,주문취소여부,주문시간,제품번호
0,1032097472,정상회원,1.0,7083,2021-01-02,NaN,NaN,0,0,주문취소,오후 12:60,100021783V2_1337
1,1032097472,정상회원,1.0,29865,2021-01-02,2021-01-02,2021-01-02,0,0,NaN,오후 12:60,100022137V2_1606
2,1032097472,정상회원,1.0,23164,2021-01-02,2021-01-02,2021-01-03,0,0,NaN,오후 12:60,100021452V2_1113
3,369152832,정상회원,1.0,16655,2021-01-02,2021-01-02,2021-01-03,0,0,NaN,오후 12:60,10002931V2_708
4,1032097472,정상회원,1.0,8423,2021-01-02,NaN,NaN,0,0,주문취소,오후 12:60,100022085V2_1559


In [261]:
df_member.head()

,Unnamed: 0,회원번호,회원상태,성별,나이,등록카드,결혼,구독여부,주소지,세부주소지
0,0,18764160,정상회원,여,68,농협중앙회,기혼,False,서울특별시,성동구
1,1,18792000,정상회원,남,83,연결앱결제,NaN,False,강원도,강릉시
2,2,18942336,정상회원,여,39,신한은행,기혼,False,인천광역시,중구
3,3,18949760,정상회원,여,73,기업은행,NaN,NaN,강원도,홍천군
4,4,19391488,정상회원,여,52,연결앱결제,기혼,False,대전광역시,중구


In [262]:
df_product.head()

,제품번호,물품명,물품대분류,물품중분류,상품중량
0,100021V2_0,2단무늬컵,식기/편백,자기,1p
1,100022V2_1,7곡딸기롤과자,과자,스낵,100g/10개입
2,100023V2_2,7곡참식,식사대용,선식/생식,700g
3,100024V2_3,가리비,생물수산,패류/갑각류,1.5kg
4,100025V2_3,가리비,생물수산,패류/갑각류,1kg


## 재구매율

재구매율 = (두 번 이상 구매한 고객 수/전체 고객 수) * 100

In [268]:
# '주문일시' 열을 날짜 형식으로 변환
df_sales['주문일시'] = pd.to_datetime(df_sales['주문일시'])

# '회원번호'별 구매 횟수 계산
df_sales['구매횟수'] = 1
customer_order_count = df_sales.pivot_table(
    index=['회원번호','제품번호'],values='구매횟수',aggfunc='sum')

customer_order_count

구매횟수
회원번호       제품번호                 
18764160   100021182V2_892     1
           100021211V2_918     1
           100021235V2_939     1
           10002134V2_106      1
           10002158V2_128      1
...                          ...
1670620864 10002805V2_612      1
           10002819V2_621      1
           10002869V2_665      1
           10002897V2_680      1
           1000296V2_74        2

[373551 rows x 1 columns]

In [294]:
# '구매횟수'가 2 이상인 고객 수 계산
customer_order_count_2 = customer_order_count[customer_order_count['구매횟수'] >= 2].index.get_level_values('회원번호').nunique()

customer_order_count_2

7997

In [296]:
# 전체 고객 수 
customer_total = df_sales['회원번호'].value_counts().shape[0]

# 재구매율 계산
re_order_rate = (customer_order_count_2 / customer_total) * 100
print('재구매율:', round(re_order_rate, 2), '%')



재구매율: 63.77 %


## 고객 이탈률

- 회원상태 : 정상회원/탈퇴
- 고객 이탈률 = (탈퇴 회원 수 / 전체 회원 수) * 100

In [185]:
# 탈퇴 회원 수
member_out = df_member[df_member['회원상태'] == '탈퇴'].shape[0]
print('탈퇴 회원 수:', member_out)

# 전체 회원 수
member_total = df_member['회원상태'].shape[0]
print('전체 회원 수:', member_total)

# 고객 이탈률
out_rate = (member_out/member_total) * 100
print('고객 이탈률: ', round(out_rate,2), '%')

탈퇴 회원 수: 156
전체 회원 수: 12540
고객 이탈률:  1.24 %


## 배송 지연율

- 당일배송이 아닐 경우 배송이 지연되었다고 판단
- 배송 지연율 = (지연된 배송 건수 / 전체 배송 건수) * 100

In [325]:
# '배송시작일', '배송완료일' 열을 날짜 형식으로 변환
df_sales['배송시작일'] = pd.to_datetime(df_sales['배송시작일'])
df_sales['배송완료일'] = pd.to_datetime(df_sales['배송완료일'])

In [351]:
# '배송지연' 열을 추가하여 '배송시작일'과 '배송완료일'이 다를 경우 '배송지연'에 True 로 표시
df_sales['배송지연'] = (df_sales['배송시작일'] != df_sales['배송완료일'])

#'배송시작일','배송완료일' 결측값 제거
df_sales_clean = df_sales.dropna(subset=['배송시작일','배송완료일'])

df_sales_clean

,회원번호,회원상태,구매수량,구매금액,주문일시,배송시작일,배송완료일,사용 적립금,사용 포인트 네이버,주문취소여부,주문시간,제품번호,구매횟수,배송지연
1,1032097472,정상회원,1.0,29865,2021-01-02,2021-01-02,2021-01-02,0,0,NaN,오후 12:60,100022137V2_1606,1,False
2,1032097472,정상회원,1.0,23164,2021-01-02,2021-01-02,2021-01-03,0,0,NaN,오후 12:60,100021452V2_1113,1,True
3,369152832,정상회원,1.0,16655,2021-01-02,2021-01-02,2021-01-03,0,0,NaN,오후 12:60,10002931V2_708,1,True
7,369152832,정상회원,1.0,2489,2021-01-02,2021-01-02,2021-01-02,0,0,NaN,오후 12:60,100022160V2_1623,1,False
8,441813376,정상회원,2.0,19144,2021-01-03,2021-01-05,2021-01-05,0,0,NaN,오후 12:60,100021509V2_1149,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
668106,1523874368,정상회원,1.0,166554,2021-10-19,2021-10-20,2021-10-20,0,0,NaN,오전 01:00,100022543V2_1912,1,False
668107,402835520,정상회원,1.0,12635,2021-10-23,2021-10-23,2021-10-24,0,0,NaN,오전 01:00,10002432V2_327,1,True
668108,402835520,정상회원,1.0,7849,2021-10-23,2021-10-23,2021-10-23,0,0,NaN,오전 01:00,100022317V2_1725,1,False
668109,1544381312,정상회원,1.0,7275,2021-10-27,2021-10-27,2021-10-28,6270,0,NaN,오전 01:00,10002723V2_563,1,True


In [357]:
# '배송지연' 건수 계산
dealy_order = (df_sales_clean['배송지연'] == True).sum()

print ('배송지연건수:', dealy_order)

배송지연건수: 340876


In [363]:
# 전체 배송 건수 계산
total_order = df_sales_clean['배송지연'].shape[0]
print ('전체배송건수:', total_order)

# 배송 지연률 계산
dealy_order_rate = (dealy_order / total_order) * 100
print ('배송지연률:',dealy_order_rate,'%')

전체배송건수: 637476
배송지연률: 53.47275819011226 %
